<center> <h1> HOMO-LUMO Gap Energy of the Conjugated Dye Analysis using Python</h1> </center>

### Overview
1. [Introduction](#1.-Introduction)
2. [Minimal Background for Python](#2.-Minimal-Background-for-Python)
3. [Data Analysis](#3.-Data-Analysis)

---

## 1. Introduction

This page gives a walk through for analyzing the conjugated dye with particle in 1D box model using python. 

It should be pointed out that all the input data showed in this page do not correspond to any molecule used in the experiment, and you should plug in your own data to get result.

#### (a) Theory HOMO-LUMO Gap Energy:
 <br>
$$
\Delta E_{\text {theo}}=\frac{h^{2}\left(n_{2}^{2}-n_{1}^{2}\right)}{8 m L^{2}}=\frac{h^{2}\left(\left(\frac{N}{2}+1\right)^{2}-\left(\frac{N}{2}\right)^{2}\right)}{8 m L^{2}}=\frac{h^{2}(N+1)}{8 m L^{2}}
$$
<p style="text-align: center; font-size: 13px; font-family: times">
    N: number of π electrons<br>
    L: Chain length, may be taken as the sum of the bond lengths of all the bonds between the two "terminal" nitrogen atoms
</p>

Import $\alpha$ as a correction to L

$$
\Delta E_{c o r r}=\frac{h^{2}(N+1)}{8 m(L+\alpha)^{2}}
$$

#### (b) Experiment HOMO-LUMO Gap Energy:

$$
\Delta E_{exp}=h \nu=\frac{h c}{\lambda}
$$

<p style="color:red"><b>The goal is to quantify $\alpha$, using the data of N (num of electrons), L (chain length) and  $\lambda$ (experiment wavelength)</b></p>

---

## 2. [Minimal Background for Python](https://yyrcd-1256568788.cos.na-siliconvalley.myqcloud.com/yyrcd/2019-10-18-Minimal%20Background%20for%20Python-1.html)

1. Basic data types
2. Containers
3. Control Flow
4. Functions
5. Numpy
6. Matplotlib
7. Scipy


---

## 3. Data Analysis

In [1]:
import numpy as np
from scipy.optimize import minimize

In [2]:
average_bond_C_C = 1.39  # Angstroms
single_bond_C_N = 1.47  # Angstroms
double_bond_C_N = 1.28  # Angstroms

In [12]:
# Define some constants
h = 6.62607e-34     # planck's constant in J
me = 9.1093837e-31  # mass of an electron in kg
c = 2.99792458e8    # speed of light  m/s
J2eV = 6.242e18     # convert J -> ev, usage: J * J2eV = eV

Plug in you own data at below

In [13]:
set_1_num_C_C = np.array([5, 7, 9])
set_1_num_electron = np.array([6, 8, 10])
set_1_exp_wavelength = np.array([443.0, 527.0, 630.0])  # nm

set_2_num_C_C = np.array([5, 7, 9, 10])
set_2_num_electron = np.array([6, 8, 10, 12])
set_2_exp_wavelength = np.array([443.0, 527.0, 630.0, 720.0])  # nm

In [14]:
def get_theory_L(num_C_C):
    L = average_bond_C_C * num_C_C + single_bond_C_N + double_bond_C_N
    return L

In [17]:
# Default of a is 0, which is theory energy (not corrected)
def get_theory_E(N, L, a=0):
    a = a * 1e-10  # convert Angstroms -> m
    L = L * 1e-10  # convert Angstroms -> m
    e = h * h * (N + 1) / (8 * me * (L + a)**2 )
    e = e * J2eV  # convert J -> eV
    return e

In [18]:
def get_E_from_wavelength(wavelength):
    wavelength = wavelength * 1e-9  # convert nm -> m
    e = h * c / wavelength
    e = e * J2eV  # convert J -> eV
    return e

def get_wavelength_from_E(e):
    e = e / J2eV  # convert eV -> J
    wavelength = h * c / e
    wavelength = wavelength * 1e9  # convert m -> nm
    return wavelength

In [19]:
def fit(num_C_C, num_electron, exp_wavelength):

    # Theory Energy
    theo_L = get_theory_L(num_C_C)
    theo_E = get_theory_E(num_electron, theo_L)
    theo_wavelength = get_wavelength_from_E(theo_E)
    
    # experiment Energy
    exp_E = get_E_from_wavelength(exp_wavelength)

    def loss(para):
        pred_E = get_theory_E(num_electron, theo_L, a=para[0])
        mse = np.mean((pred_E - exp_E)**2)  # mean_squared_error
        return mse

    result = minimize(loss, x0=np.array([1.0]), method='Nelder-Mead')
    para = result.x
    theo_E_corrected = get_theory_E(num_electron, theo_L, a=para[0])

    print('Theory Wavelength    (nm): {}'.format(theo_wavelength))
    print('Experi Wavelength    (nm): {}'.format(exp_wavelength))
    print()
    print('Theory Energy        (eV): {}'.format(theo_E))
    print('Experi Energy        (eV): {}'.format(exp_E))
    print('Fitted a                 : {:.3f}'.format(para[0]))
    print('Theory Energy (corrected): {}'.format(theo_E_corrected))

In [20]:
fit(set_1_num_electron, set_1_num_C_C, set_1_exp_wavelength)

Theory Wavelength    (nm): [675.8580762  792.87842608 914.05460624]
Experi Wavelength    (nm): [443. 527. 630.]

Theory Energy        (eV): [1.83461516 1.56384565 1.3565267 ]
Experi Energy        (eV): [2.79896044 2.35282633 1.9681579 ]
Fitted a                 : -2.295
Theory Energy (corrected): [2.91675355 2.24531337 1.82485337]


In [21]:
fit(set_2_num_electron, set_2_num_C_C, set_2_exp_wavelength)

Theory Wavelength    (nm): [ 675.8580762   792.87842608  914.05460624 1131.60949259]
Experi Wavelength    (nm): [443. 527. 630. 720.]

Theory Energy        (eV): [1.83461516 1.56384565 1.3565267  1.09573089]
Experi Energy        (eV): [2.79896044 2.35282633 1.9681579  1.72213816]
Fitted a                 : -2.368
Theory Energy (corrected): [2.96587842 2.27396079 1.84359272 1.42094771]
